#1. Import library



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#2. Import data

In [ ]:
metadata_movie = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")

/tmp/ipykernel_25/4214782242.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_movie = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")


#3.Data proprocessing

In [ ]:
data_movie = metadata_movie[["id","title","genres","overview"]]

In [ ]:
def compute_len_string(x):
  if isinstance(x,str):
    return len(x)
  else:
    return 0

In [ ]:
data_movie["len_overview"] = data_movie["overview"].apply(compute_len_string)
#After experiment. i find that to drop None value overview or unknown overview, i will drop overview with length of string < 20
data_movie[data_movie["len_overview"] < 20] = None
data_movie[data_movie["genres"] == "[]"] = None
data_movie.dropna(inplace = True)
data_movie.drop(columns = ["len_overview"],inplace = True)

#drop duplicate
data_movie.drop_duplicates(inplace = True)

data_movie["id"] = data_movie["id"].astype(np.int64)
data_movie

/tmp/ipykernel_25/3699323544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movie["len_overview"] = data_movie["overview"].apply(compute_len_string)
/tmp/ipykernel_25/3699323544.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movie[data_movie["len_overview"] < 20] = None
/tmp/ipykernel_25/3699323544.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,id,title,genres,overview
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...
...,...,...,...,...
45459,222848,Caged Heat 3000,"[{'id': 878, 'name': 'Science Fiction'}]",It's the year 3000 AD. The world's most danger...
45460,30840,Robin Hood,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...","Yet another version of the classic epic, with ..."
45461,439050,Subdue,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",Rising and falling between a man and woman.
45462,111109,Century of Birthing,"[{'id': 18, 'name': 'Drama'}]",An artist struggles to finish his work while a...


##3.1 Transform genres

In [ ]:
import ast
def transforms_genres(x):
  x_list = ast.literal_eval(x)
  if x_list == []:
    return None

  list_attr = []
  for dic in x_list:
    list_attr.append(dic["name"].lower())
  return list(set(list_attr))

genre = data_movie["genres"]
genre = genre.apply(transforms_genres)
data_movie["genres"] = genre
data_movie = data_movie.dropna()

/tmp/ipykernel_25/793517020.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_movie["genres"] = genre


##3.2 Lowercase overview and title

In [ ]:
#lower all the attribute:
data_movie["title"] = data_movie["title"].str.lower()
data_movie["overview"] = data_movie["overview"].str.lower()

In [ ]:
data_movie_tfidf = data_movie.copy(deep =True)
data_movie_tfidf.index = range(0, len(data_movie_tfidf.index))

In [ ]:
data_movie

,id,title,genres,overview
0,862,toy story,"[family, comedy, animation]","led by woody, andy's toys live happily in his ..."
1,8844,jumanji,"[fantasy, adventure, family]",when siblings judy and peter discover an encha...
2,15602,grumpier old men,"[romance, comedy]",a family wedding reignites the ancient feud be...
3,31357,waiting to exhale,"[romance, drama, comedy]","cheated on, mistreated and stepped on, the wom..."
4,11862,father of the bride part ii,[comedy],just when george banks has recovered from his ...
...,...,...,...,...
45459,222848,caged heat 3000,[science fiction],it's the year 3000 ad. the world's most danger...
45460,30840,robin hood,"[romance, action, drama]","yet another version of the classic epic, with ..."
45461,439050,subdue,"[family, drama]",rising and falling between a man and woman.
45462,111109,century of birthing,[drama],an artist struggles to finish his work while a...


##3.3 Tokenize overview and title

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Tokenize the overview and title
data_movie["overview"] = data_movie["overview"].apply(word_tokenize)
data_movie["title_list"] = data_movie["title"].apply(word_tokenize)

##3.4 Removing Punctuations and Stopwords


In [ ]:
from nltk.corpus import stopwords
from string import punctuation

list_stopwords = set(stopwords.words('english') + list(punctuation))
data_movie["overview"] = data_movie["overview"].apply(lambda x: [word for word in x if word not in list_stopwords])
data_movie["title_list"] = data_movie["title_list"].apply(lambda x: [word for word in x if word not in list_stopwords])
data_movie["genres"] = data_movie["genres"].apply(lambda x: [word for word in x if word not in list_stopwords])

In [ ]:
import string
data_movie['overview'] = data_movie['overview'].apply(lambda x : [word.translate(str.maketrans('', '', string.punctuation)) for word in x])
data_movie['overview'] = data_movie['overview'].apply(lambda x : [word for word in x if len(word) > 0])

In [ ]:
data_movie["title_list"] = data_movie["title_list"].apply(lambda x : list(set(x)))
data_movie["genres"] = data_movie["genres"].apply(lambda x : list(set(x)))
data_movie['overview'] = data_movie['overview'].apply(lambda x : list(set(x)))

##3.5 Import ratings data and split into train and test

In [ ]:
ratings_ori = pd.read_csv("/kaggle/input/the-movies-dataset/ratings.csv")
movie_id = list(data_movie_tfidf["id"])
ratings = ratings_ori[ratings_ori["movieId"].isin(movie_id)]
ratings.index = range(0,len(ratings.index))

In [ ]:
class Datasets:
  def __init__(self, ratings : pd.DataFrame, movie_data: pd.DataFrame):
    self.ratings = ratings
    self.movie_data = movie_data

  def split_data(self):
    self.ratings = self.ratings[self.ratings["rating"] >= 3]
    self.ratings.index = range(0, len(self.ratings.index))
    self.ratings["rating_order"] = self.ratings.groupby("userId")["timestamp"].rank(ascending = True, method = "first")
    # We split selk.No_item_test for tesing, n - self.No_item_test for training
    self.rating_train = self.ratings[self.ratings["rating_order"] <= 1]
    self.rating_test = self.ratings[self.ratings["rating_order"] > 1]


#4.TfIdf Vectorizer

##4.1 Build tf-idf matrix

In [ ]:
tfidf_datasets = Datasets(ratings, data_movie_tfidf)
tfidf_datasets.split_data()

/tmp/ipykernel_25/152895867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ratings["rating_order"] = self.ratings.groupby("userId")["timestamp"].rank(ascending = True, method = "first")


In [ ]:
def create_doc_overview(x):
    return x["overview"]

In [ ]:
data_tfidf = tfidf_datasets.movie_data.apply(create_doc_overview,axis = 1)

#Initialize tfidf object with removing ll english word such as: 'the',......
tfidf = TfidfVectorizer(stop_words = 'english')


#Fit the tfidf object for data column "overview"
tfidf_matrix_movie = tfidf.fit_transform(data_tfidf)


In [ ]:
def get_recommendations_title(title, tfidf_matrix_movie,data_movie_tfidf,number_recommendation):
    # Get the index of the movie user like in rating train
    movieId_index = pd.Series(data_movie_tfidf.index, index = data_movie_tfidf["id"].astype('int64'))
    movieId = int(data_movie_tfidf[data_movie_tfidf["title"] == title]["id"])

    #Computes the cosine similarity between the user profile and all item profiles
    cosine_similarities = cosine_similarity(tfidf_matrix_movie[movieId_index[movieId]], tfidf_matrix_movie)[0]
    #Gets the top similar items index
    similar_indices = cosine_similarities.argsort()[-2:-2-number_recommendation:-1]

    # Return the top 10 most similar movies
    return data_movie_tfidf['title'].iloc[similar_indices]

In [ ]:
get_recommendations_title("toy story", tfidf_matrix_movie,tfidf_datasets.movie_data,10)

/tmp/ipykernel_25/3529450204.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movieId = int(data_movie_tfidf[data_movie_tfidf["title"] == title]["id"])


15011                                     toy story 3
2904                                      toy story 2
23588                                       small fry
10095                          the 40 year old virgin
22963                     andy hardy's blonde trouble
35946    superstar: the life and times of andy warhol
39733    andy peters: exclamation mark question point
8147                                        the champ
1022                            rebel without a cause
26087                      life begins for andy hardy
Name: title, dtype: object

In [ ]:
def get_recommendations_index(user_id, rating_train,tfidf_matrix_movie,movie_data,number_recommendation):
    # Get the index of the movie user like in rating train
    movieId_index = pd.Series(movie_data.index, index = movie_data["id"].astype('int64'))
    movie_idx = [int(movieId) for movieId in rating_train[rating_train["userId"] == user_id]["movieId"]]

    #Computes the cosine similarity between the user profile and all item profiles
    cosine_similarities = cosine_similarity(tfidf_matrix_movie[movieId_index[movie_idx]], tfidf_matrix_movie)[0]
    #Gets the top similar items index
    similar_indices = cosine_similarities.argsort()[-2:-2-number_recommendation:-1]
    return similar_indices

##4.4 Using K-means

In [ ]:
class Kmean_TFIDF():
  def __init__(self,noOfCluster,datasets):
    self.datasets = datasets
    self.movieId_index = pd.Series(self.datasets.movie_data.index, index = self.datasets.movie_data["id"].astype('int64'))
    self.tfidf_matrix = self.init_matrix_tfidf()
    self.kmeans = cluster.KMeans(n_clusters=noOfCluster, random_state=0, n_init="auto").fit(self.tfidf_matrix)
    self.cluster_map = self.create_cluster_map()
    self.user_id = set(self.datasets.rating_train["userId"])


  def init_matrix_tfidf(self):
    data_tfidf = self.datasets.movie_data.apply(create_doc_overview,axis = 1)
    #Initialize tfidf object with removing ll english word such as: 'the',......
    tfidf = TfidfVectorizer(stop_words = 'english')

    #Fit the tfidf object for data column "overview":
    tfidf_matrix = tfidf.fit_transform(data_tfidf)

    return tfidf_matrix

  def create_cluster_map(self):
    cluster_map = pd.DataFrame()
    cluster_map['movie_index'] = self.datasets.movie_data.id.values
    cluster_map['title'] = self.datasets.movie_data.title.values
    cluster_map['cluster'] = self.kmeans.labels_
    return cluster_map

  def recommend(self,userId):
    # uv = user_vector(userId,self.ratings,self.movie_data)
    if userId in self.user_id:
      movie_id = int(self.datasets.rating_train[self.datasets.rating_train["userId"] == userId]["movieId"])
      index = self.movieId_index[movie_id]
      vector = self.tfidf_matrix[index]
      cluster_prediction = int(self.kmeans.predict(vector))
      return self.cluster_map[self.cluster_map["cluster"] == cluster_prediction]["movie_index"].to_list()
    else:
      # print(f"{userId} not in datasets")
      return None





In [ ]:
kmeans_tfidf = Kmean_TFIDF(100,tfidf_datasets)

#5.Word2Vec

##5.1 Download pre-train model

In [ ]:
import gensim
import gensim.downloader

wv = gensim.downloader.load('word2vec-google-news-300')

[===============================================---] 95.0% 1579.7/1662.8MB downloaded

##5.2 Keep only token which is in word2vec

In [ ]:
data_movie_wv = data_movie.loc[:,["id","title","genres","overview","title_list"]]

In [ ]:
data_movie_wv

,id,title,genres,overview,title_list
0,862,toy story,"[family, comedy, animation]","[aside, led, circumstances, place, duo, brings...","[toy, story]"
1,8844,jumanji,"[fantasy, adventure, family]","[door, discover, magical, find, trapped, finis...",[jumanji]
2,15602,grumpier old men,"[romance, comedy]","[divorcée, locals, ancient, restaurant, feud, ...","[men, grumpier, old]"
3,31357,waiting to exhale,"[romance, comedy, drama]","[lovers, find, better, breath, mistreated, les...","[exhale, waiting]"
4,11862,father of the bride part ii,[comedy],"[news, pregnant, receives, expecting, recovere...","[father, part, ii, bride]"
...,...,...,...,...,...
45459,222848,caged heat 3000,[science fiction],"[defenses, finds, caged, soon, remote, war, he...","[caged, heat, 3000]"
45460,30840,robin hood,"[romance, drama, action]","[darker, marian, somewhat, version, story, cha...","[robin, hood]"
45461,439050,subdue,"[family, drama]","[falling, rising, man, woman]",[subdue]
45462,111109,century of birthing,[drama],"[cult, storyline, artist, plays, head, finish,...","[birthing, century]"


In [ ]:
matrix_film_vocab = []
for list_ in data_movie_wv.to_numpy():
    list_[2] = [word for word in list_[2] if word in wv.key_to_index]
    list_[3] = [word for word in list_[3] if word in wv.key_to_index]
    list_[4] = [word for word in list_[4] if word in wv.key_to_index]
    matrix_film_vocab.append(list_)
movie_vocab = pd.DataFrame(matrix_film_vocab, columns=data_movie_wv.columns)

In [ ]:
datasets_w2v = Datasets(ratings,data_movie)
datasets_w2v.split_data()

/tmp/ipykernel_25/152895867.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ratings["rating_order"] = self.ratings.groupby("userId")["timestamp"].rank(ascending = True, method = "first")


##5.4 Recommendation using cosine similarity

In [ ]:
number_recommendation_test_datasets = datasets_w2v.rating_test.groupby("userId").count()["rating"]

In [ ]:
def recommendation_user(user_id, rating_train, data_movie_wv,number_recommendation):
    movieId_data_prefer_by_user = [int(movieId) for movieId in rating_train[rating_train["userId"] == user_id]["movieId"]]
    data_title_vocab = []
    # ratings_movie = [rating for rating in rating_train[rating_train["userId"] == user_id]["rating"]]
    for movieId in movieId_data_prefer_by_user:
          list_ = data_movie_wv[data_movie_wv["id"] == movieId].to_numpy()[0]
          list_[2] = [word for word in list_[2] if word in wv.key_to_index]
          list_[3] = [word for word in list_[3] if word in wv.key_to_index]
          list_[4] = [word for word in list_[4] if word in wv.key_to_index]
          data_title_vocab.append(list_)

    matrix_similarity = []
    for list1 in data_title_vocab:
        for list2 in movie_vocab.to_numpy():
            try:
                score_title = wv.n_similarity(list1[4], list2[4])
                score_genre = wv.n_similarity(list1[2], list2[2])
                score_overview = wv.n_similarity(list1[3], list2[3])
            except:
                score_title = 0
                score_genre = 0
                score_genre = 0
            if ((list1[1] != list2[1]) & (score_genre > 0.85)):
                matrix_similarity.append([list2[1], list1[1], score_title, score_genre, score_overview])
    df_netflix_similarity = pd.DataFrame(matrix_similarity, columns = ['recommendation','title','score_title', 'score_genre', 'score_overview'])
    df_netflix_similarity['final_score'] = (df_netflix_similarity['score_title'] + df_netflix_similarity['score_genre'] +\
                                           df_netflix_similarity['score_overview'])
    return (df_netflix_similarity.sort_values(by=['final_score', 'score_genre', 'score_overview','score_title'], ascending=False).head(number_recommendation[user_id]))

#6.Evaluation

In [ ]:
def Precision(prediction, ground_truth):
  count = 0
  if len(prediction) == 0:
    return 0
  for movie in prediction:
    if movie in ground_truth:
      count += 1
  return count / len(prediction)

In [ ]:
def PrecisionAllUserW2V(datasets):
  rating_test_userId = set(datasets.rating_test["userId"])
  rating_train_userId = set(datasets.rating_train["userId"])
  precision = 0
  for userId in tqdm(list(rating_train_userId)[:200]):
    if userId in rating_test_userId:
      prediction  = recommendation_user(userId,datasets.rating_train, data_movie_wv,number_recommendation_test_datasets).index.to_list()
      grouth_truth = datasets.rating_test[datasets.rating_test["userId"] == userId]["movieId"].to_list()
      precision += Precision(prediction, grouth_truth)


  return precision / 200

In [ ]:
PrecisionAllUserW2V(datasets_w2v)

100%|██████████| 200/200 [1:24:47<00:00, 25.44s/it]


0.008341336357783702

In [ ]:
def PrecisionAllUserTFIDF(datasets):
  rating_test_userId = set(datasets.rating_test["userId"])
  rating_train_userId = set(datasets.rating_train["userId"])
  precision = 0
  for userId in tqdm(list(rating_train_userId)):
    if userId in rating_test_userId:
        grouth_truth = datasets.rating_test[datasets.rating_test["userId"] == userId]["movieId"].to_list()
        prediction  = get_recommendations_index(userId,datasets.rating_train,tfidf_matrix_movie,datasets.movie_data,len(grouth_truth))
        precision += Precision(prediction, grouth_truth)
  return precision / len(rating_test_userId)


In [ ]:
PrecisionAllUserTFIDF(tfidf_datasets)

100%|██████████| 262854/262854 [2:35:46<00:00, 28.12it/s]


0.0009489786558507569

In [ ]:
rating_train_userId = set(tfidf_datasets.rating_train["userId"])
precision = 0
No_user = 0
for u in tqdm(list(rating_train_userId)):
  prediction = kmeans_tfidf.recommend(u)
  if prediction:
    ground_truth = tfidf_datasets.rating_test[tfidf_datasets.rating_test["userId"] == u]["movieId"].to_list()
    precision += Precision(prediction, ground_truth)
    No_user += 1

print(precision/No_user)

  0%|          | 0/262854 [00:00<?, ?it/s]/tmp/ipykernel_25/2786642869.py:31: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movie_id = int(self.datasets.rating_train[self.datasets.rating_train["userId"] == userId]["movieId"])
100%|██████████| 262854/262854 [1:08:31<00:00, 63.94it/s]

0.0008372021587223145
